In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import pickle

from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
from google.colab import files
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import classification_report, accuracy_score


In [ ]:
with open('/right_model.pkl', 'rb') as f:
    right_model = pickle.load(f)
with open('/left_model.pkl', 'rb') as f:
    left_model = pickle.load(f)
print(type(right_model))
print(type(left_model))

<class 'sklearn.model_selection._search.GridSearchCV'>
<class 'sklearn.model_selection._search.GridSearchCV'>


In [ ]:
new_data = pd.read_csv('/Fall24.csv')

new_data = new_data.drop(['Tilt'], axis = 1)

new_data = new_data.iloc[:, 28:45].join(new_data.iloc[:, 7])

new_left = new_data[new_data['PitcherThrows'] == 'Left']
new_right = new_data[new_data['PitcherThrows'] == 'Right']

new_left = new_left.drop(['PitcherThrows'], axis = 1)
new_right = new_right.drop(['PitcherThrows'], axis = 1)

print(new_left.head())
print(new_right.head())

    RelSpeed  VertRelAngle  HorzRelAngle     SpinRate    SpinAxis  RelHeight  \
61  88.78874     -1.997445      5.240310  2202.983454  119.640679    5.29923   
62  82.75755      0.422878      2.879386  1792.017351   95.141739    5.21506   
63  89.34908     -1.816993      3.152898  2230.537081  133.693907    5.39367   
64  81.02230     -1.040396      2.015421  2595.075890  265.771883    5.52859   
65  80.81065     -0.732781      1.322497  2797.042055  268.503497    5.47950   

    RelSide  Extension  VertBreak  InducedVertBreak  HorzBreak  \
61 -1.70959    4.95732  -25.90923          11.43839  -17.75186   
62 -1.61507    4.92478  -39.48837           3.31488  -20.15387   
63 -1.52184    4.84172  -20.66706          16.46292  -15.76280   
64 -1.22644    4.77161  -42.69416           1.76311    4.94083   
65 -1.43662    4.75507  -43.29183           1.78980    9.62780   

    PlateLocHeight  PlateLocSide  ZoneSpeed  VertApprAngle  HorzApprAngle  \
61         1.24424       1.77603   80.36269  

In [ ]:
from enum import auto
pred_left = left_model.predict(new_left)
pred_right = right_model.predict(new_right)

category_mapping = {
    0: 'ChangeUp', 1: 'Curveball', 2: 'Cutter', 3: 'Fastball',
    4: 'Other', 5: 'Sinker', 6: 'Slider', 7: 'Splitter', 8: 'Undefined'
}

# Round predictions to the nearest integer
rounded_left = np.round(pred_left).astype(int)
rounded_right = np.round(pred_right).astype(int)

# Map integer predictions back to pitch names
mapped_predictions_left = [category_mapping[p] for p in rounded_left]
mapped_predictions_right = [category_mapping[p] for p in rounded_right]

print(len(mapped_predictions_left))
print(len(mapped_predictions_right))

actual_pitches_left = pd.read_csv('/Fall24.csv')
actual_pitches_left = actual_pitches_left[actual_pitches_left['PitcherThrows'] == 'Left']['TaggedPitchType']
actual_pitches_right = pd.read_csv('/Fall24.csv')
actual_pitches_right = actual_pitches_right[actual_pitches_right['PitcherThrows'] == 'Right']['TaggedPitchType']
auto_pitches_left = pd.read_csv('/Fall24.csv')
auto_pitches_left = auto_pitches_left[auto_pitches_left['PitcherThrows'] == 'Left']['AutoPitchType']
auto_pitches_right = pd.read_csv('/Fall24.csv')
auto_pitches_right = auto_pitches_right[auto_pitches_right['PitcherThrows'] == 'Right']['AutoPitchType']

print(len(actual_pitches_left))
print(len(actual_pitches_right))

# Save mapped predictions to CSV
Left_predicted_df = pd.DataFrame({
    'Actual_Left_Pitch': actual_pitches_left[:len(rounded_left)],
    'Auto_Left_Pitch': auto_pitches_left[:len(rounded_left)],
    'Predicted_Left_Pitch': [category_mapping[p] for p in rounded_left],
})

Right_predicted_df = pd.DataFrame({
    'Actual_Right_Pitch': actual_pitches_right[:len(rounded_right)],
    'Auto_Right_Pitch': auto_pitches_right[:len(rounded_right)],
    'Predicted_Right_Pitch': [category_mapping[p] for p in rounded_right],
})


Right_predicted_df.to_csv('Right_new_pitch_predictions.csv', index=False)
Left_predicted_df.to_csv('Left_new_pitch_predictions.csv', index=False)


372
1087
372
1087


In [ ]:
accurate_predictions_left = (Left_predicted_df['Actual_Left_Pitch'] == Left_predicted_df['Predicted_Left_Pitch']).sum()
total_predictions_left = len(Left_predicted_df)
accuracy_percentage_left = (accurate_predictions_left / total_predictions_left) * 100
auto_correct_left = (Left_predicted_df['Auto_Left_Pitch'] == Left_predicted_df['Predicted_Left_Pitch']).sum()
auto_correct_percentage_left = (auto_correct_left / total_predictions_left) * 100
auto_actual_left = (Left_predicted_df['Auto_Left_Pitch'] == Left_predicted_df['Actual_Left_Pitch']).sum()
auto_actual_percentage_left = (auto_actual_left / total_predictions_left) * 100

print(total_predictions_left)
print(accurate_predictions_left)
print(auto_correct_left)
print(auto_actual_left)
print(f"Left Actual vs predicted: {accuracy_percentage_left:.2f}%")
print(f"Left Auto vs predicted: {auto_correct_percentage_left:.2f}%")
print(f"Left Auto vs Actual: {auto_actual_percentage_left:.2f}%")

372
353
63
65
Left Actual vs predicted: 94.89%
Left Auto vs predicted: 16.94%
Left Auto vs Actual: 17.47%


In [ ]:
accurate_predictions_right = (Right_predicted_df['Actual_Right_Pitch'] == Right_predicted_df['Predicted_Right_Pitch']).sum()
total_predictions_right = len(Right_predicted_df)
accuracy_percentage_right = (accurate_predictions_right / total_predictions_right) * 100
auto_correct_right = (Right_predicted_df['Auto_Right_Pitch'] == Right_predicted_df['Predicted_Right_Pitch']).sum()
auto_correct_percentage_right = (auto_correct_right / total_predictions_right) * 100
auto_actual_right = (Right_predicted_df['Auto_Right_Pitch'] == Right_predicted_df['Actual_Right_Pitch']).sum()
auto_actual_percentage_right = (auto_actual_right / total_predictions_right) * 100

print(total_predictions_right)
print(accurate_predictions_right)
print(auto_correct_right)
print(auto_actual_right)
print(f"Right Actual vs predicted: {accuracy_percentage_right:.2f}%")
print(f"Right Auto vs predicted: {auto_correct_percentage_right:.2f}%")
print(f"Right Auto vs Actual: {auto_actual_percentage_right:.2f}%")

1087
909
236
298
Right Actual vs predicted: 83.62%
Right Auto vs predicted: 21.71%
Right Auto vs Actual: 27.41%


In [ ]:
files.download('Right_new_pitch_predictions.csv')
files.download('Left_new_pitch_predictions.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>